# Exploring COVID-19 time series data from Johns Hopkins University

This notebook analyzes the data in the [CSSEGISandData/COVID-19](https://github.com/CSSEGISandData/COVID-19) github repo. This assumes you have cloned the repo at the local path `~/github/CSSEGISandData/COVID-19`.

In [ ]:
import os
import pandas as pd
import torch
import datetime

In [ ]:
dirname = os.path.expanduser("~/github/CSSEGISandData/COVID-19/"
                             "csse_covid_19_data/csse_covid_19_time_series")
def read_csv(basename):
    return pd.read_csv(os.path.join(dirname, basename), header=0)
us_cases_df = read_csv("time_series_covid19_confirmed_US.csv")
us_deaths_df = read_csv("time_series_covid19_deaths_US.csv")
global_cases_df = read_csv("time_series_covid19_confirmed_global.csv")
global_deaths_df = read_csv("time_series_covid19_deaths_global.csv")

In [ ]:
us_cases_df

In [ ]:
us_deaths_df

In [ ]:
global_cases_df

In [ ]:
global_deaths_df

In [ ]:
print(us_cases_df.columns[6:])
print(us_deaths_df.columns[6:])
print(global_cases_df.columns)
print(global_deaths_df.columns)

In [ ]:
def to_torch(df, first_column):
    df = df[df.columns[first_column:]]
    return torch.from_numpy(df.to_numpy()).float()

case_data = torch.cat([to_torch(us_cases_df, first_column=11),
                       to_torch(global_cases_df, first_column=4)])
print(case_data.shape)
case_data

In [ ]:
death_data = torch.cat([to_torch(us_deaths_df, first_column=12),
                        to_torch(global_deaths_df, first_column=4)])
print(death_data.shape)
death_data

In [ ]:
import pickle
gisaid_stats = pickle.load(open("results/gisaid.stats.pkl", "rb"))
print(gisaid_stats.keys())

In [ ]:
for name, count in gisaid_stats["location"].most_common(20):
    print(f"{count}\t{name}")

In [ ]:
gisaid_locations = {tuple(part.strip() for part in key.lower().split("/")[1:])
                    for key in gisaid_stats["location"]}
gisaid_locations = {loc for loc in gisaid_locations if len(loc) >= 1}
print(len(gisaid_locations))
print(sorted(gisaid_locations)[:20])

In [ ]:
jhu_locations = set()
for i, row in us_cases_df[["Country_Region", "Province_State", "Admin2"]].iterrows():
    a, b, c = row
    if isinstance(c, str):
        jhu_locations.add((a.lower(), b.lower(), c.lower()))
    else:
        jhu_locations.add((a.lower(), b.lower()))
for i, row in global_cases_df[["Country/Region", "Province/State"]].iterrows():
    a, b = row
    if isinstance(b, str):
        jhu_locations.add((a.lower(), b.lower()))
    else:
        jhu_locations.add((a.lower(),))
assert len(jhu_locations) == len(us_cases_df) + len(global_cases_df)
print(len(jhu_locations))
print(sorted(jhu_locations)[:20])

In [ ]:
print(len(gisaid_locations & jhu_locations))
print(len(gisaid_locations - jhu_locations))
print(len(jhu_locations - gisaid_locations))

In [ ]:
gc = {loc[0] for loc in gisaid_locations}
jc = {loc[0] for loc in jhu_locations}
print(gc - jc)

In [ ]:
{row for row in jhu_locations if any("french" in col for col in row)}

In [ ]:
gisaid_to_jhu = {
    "aruba": ("netherlands", "aruba"),
    "bermuda": ("united kingdom", "bermuda"),
    "crimea": ("ukraine",),  # or "russia"?
    "curacao": ("netherlands", "curacao"),
    "czech repubic": ("czechia",),
    "czech republic": ("czechia",),
    "côte d'ivoire": ("cote d'ivoire",),
    "democratic republic of the congo": ("congo (kinshasa)",),
    "faroe islands": ("denmark", "faroe islands"),
    "gibraltar": ("united kingdom", "gibraltar"),
    "guadeloupe": ("france", "guadeloupe"),
    "guam": ("us", "guam"),
    "guyane francaise": ("france", "french guiana"),
    "hong kong": ("china", "hong kong"),
    "mayotte": ("france", "mayotte"),
    "myanmar": ("burma",),
    "palestine": ("israel",),  # ?
    "republic of congo": ("congo (brazzaville)",),
    "reunion": ("france", "reunion"),
    "saint barthélemy": ("france", "saint barthelemy"),
    "saint martin": ("france", "st martin"),
    "south korea": ("korea, south",),
    "st eustatius": ("netherlands", "bonaire, sint eustatius and saba"),
    "st. lucia": ("saint lucia",),
    "taiwan": ("china",),  # ?
    "trinidad": ("trinidad and tobago",),
    "usa": ("us",),
    "viet nam": ("vietnam",),
}
for country in gc - jc:
    c = gisaid_to_jhu[country]
    assert c is None or isinstance(c, tuple), c

## Joining with population data from UN

`WPP2019_TotalPopulationBySex.csv` was downloaded from https://population.un.org/wpp/Download/Standard/CSV/

In [ ]:
df = pd.read_csv("data/WPP2019_TotalPopulationBySex.csv", header=0)
df.columns

In [ ]:
df = df[df["Time"] == 2020]
df = df[df["Variant"] == "High"]

In [ ]:
uc = {name.lower() for name in df["Location"].to_list()}
jc - uc

In [ ]:
{row for row in uc if "myanmar" in row}

In [ ]:
jhu_to_un = {
    "bolivia": "bolivia (plurinational state of)",
    "brunei": "brunei darussalam",
    "burma": "myanmar",
    "congo (brazzaville)": "congo",
    "congo (kinshasa)": "democratic republic of the congo",
    "cote d'ivoire": "côte d'ivoire",
    "diamond princess": None,  # cruise ship
    "iran": "iran (islamic republic of)",
    "korea, south": "republic of korea",
    "kosovo": "serbia",
    "laos": "lao people's democratic republic",
    "moldova": "republic of moldova",
    "ms zaandam": None,  # cruise ship
    "russia": "russian federation",
    "syria": "syrian arab republic",
    "taiwan*": "china, taiwan province of china",
    "tanzania": "united republic of tanzania",
    "us": "united states of america",
    "venezuela": "venezuela (bolivarian republic of)",
    "vietnam": "viet nam",
    "west bank and gaza": "israel",
}

In [ ]:
for c in jc:
    if c not in uc:
        c2 = jhu_to_un[c]
        if c2 is not None:
            assert c2 in uc, (c, c2)